In [55]:
import pandas as pd
import numpy as np

In [56]:
df=pd.read_csv(r'/content/train.csv')

In [57]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [58]:
df.drop(['Name','PassengerId','Ticket','SibSp','Cabin','Embarked','Parch'],axis=1,inplace=True)

In [59]:
df.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500


In [60]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
Fare          0
dtype: int64

In [61]:
df['Age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [62]:
df['Age'].fillna(df['Age'].mean(),inplace=True)

In [63]:
df.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
Fare        0
dtype: int64

In [64]:
sex_dummies=pd.get_dummies(df['Sex'],drop_first=True)

In [65]:
df=pd.concat([df,sex_dummies],axis=1)

In [66]:
df.head()

,Survived,Pclass,Sex,Age,Fare,male
0,0,3,male,22.0,7.2500,1
1,1,1,female,38.0,71.2833,0
2,1,3,female,26.0,7.9250,0
3,1,1,female,35.0,53.1000,0
4,0,3,male,35.0,8.0500,1


In [67]:
df.drop(['Sex'],axis=1,inplace=True)

In [68]:
df.head()

,Survived,Pclass,Age,Fare,male
0,0,3,22.0,7.2500,1
1,1,1,38.0,71.2833,0
2,1,3,26.0,7.9250,0
3,1,1,35.0,53.1000,0
4,0,3,35.0,8.0500,1


In [69]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

In [70]:
feature_scale=['Age','Fare']
df[feature_scale]=sc.fit_transform(df[feature_scale])

In [71]:
df.head()

,Survived,Pclass,Age,Fare,male
0,0,3,-0.592481,-0.502445,1
1,1,1,0.638789,0.786845,0
2,1,3,-0.284663,-0.488854,0
3,1,1,0.407926,0.420730,0
4,0,3,0.407926,-0.486337,1


In [72]:
x=df.drop(['Survived'],axis=1)
y=df['Survived']


In [73]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [74]:
model_param = {
    'DecisionTreeClassifier':{
        'model':DecisionTreeClassifier(),
        'param':{
            'criterion': ['gini','entropy']
        }
    },
        'KNeighborsClassifier':{
        'model':KNeighborsClassifier(),
        'param':{
            'n_neighbors': [5,10,15,20,25]
        }
    },
        'SVC':{
        'model':SVC(),
        'param':{
            'kernel':['rbf','linear','sigmoid'],
            'C': [0.1, 1, 10, 100]
         
        }
    }
}

In [ ]:
scores =[]
for model_name, mp in model_param.items():
    model_selection = GridSearchCV(estimator=mp['model'],param_grid=mp['param'],cv=5,return_train_score=False)
    model_selection.fit(x,y)
    scores.append({
        'model': model_name,
        'best_score': model_selection.best_score_,
        'best_params': model_selection.best_params_
    })

In [ ]:
df_model_score = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df_model_score

In [ ]:
model_svc = SVC( C= 100,kernel='rbf')

In [ ]:
model_svc.fit(x, y)

In [ ]:
df1=pd.read_csv(r'/content/test.csv')

In [ ]:
df1.head()

In [ ]:
df2=df1.drop(['PassengerId','Name','Ticket','Cabin','Embarked','SibSp','Parch'], axis=1)

In [ ]:
df1.isnull().sum()

Pclass     0
Sex        0
Age       86
Fare       1
dtype: int64

In [ ]:
df1['Age'].fillna(df1['Age'].mean(),inplace=True)
df1['Fare'].fillna(df1['Fare'].mean(),inplace=True)

In [ ]:
sex_dummies=pd.get_dummies(df1['Sex'],drop_first=True)
df1= pd.concat([df1,sex_dummies],axis=1)
df1.drop(['Sex'], axis=1, inplace=True )

In [ ]:
df1.head()

In [ ]:
df1[feature_scale] = sc.fit_transform(df1[feature_scale])

In [ ]:
df1.head()

In [ ]:
y_predicted = model_svc.predict(df1)

In [ ]:
submission = pd.DataFrame({
        "PassengerId": df1['PassengerId'],
        "Survived": y_predicted
    })